In [ ]:
import yaml
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, max, min, count, broadcast, desc, asc, when, rank, round
from pyspark.sql.window import Window
from pyspark.sql.types import StringType, IntegerType, FloatType

In [2]:
def control_null (dataframe): 
    for dfcol in dataframe.columns:
        df_null = dataframe.filter(col(dfcol).isNull()).count()
        if df_null > 0:
            print(f"{dfcol} : {df_null} null")
        else:
            print(f"{dfcol} no tiene null")

In [3]:
spark = (
    SparkSession.builder
    .appName("Spark")
    .config("spark.driver.extraJavaOptions", r'-Dlog4j.configurationFile=file:/home/illidan/proyecto_desde0/ETL/log4j.properties')\
    .getOrCreate()
)

spark.sparkContext.setLogLevel("INFO")

logger = spark._jvm.org.apache.log4j.LogManager.getLogger(__name__)

logger.info("Log de ejemplo guardado en archivo y consola.")

errores_detectados = []


your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


14:53:21.719 [Thread-3] INFO  __main__ - Log de ejemplo guardado en archivo y consola.


In [4]:
try:
    #reading the config file
    #geting file path into a dictionary
    with open("/home/illidan/proyecto_desde0/Config_file/Config.Yaml", "r") as file:
        config = yaml.safe_load(file)
except Exception as error:
    logger.error("Error ruta .yaml:" + str(error))
    
try:
    read_parquet_airline = config["Parquet_file"]["df_airline"]
    read_parquet_flights = config["Parquet_file"]["df_flights"]
    read_parquet_airports = config["Parquet_file"]["df_airports"]
except Exception as error:
    logger.error("Error ruta .yaml:" + str(error))


In [ ]:
try:
    df_flights = spark.read.parquet(read_parquet_flights)
    df_airline = spark.read.parquet(read_parquet_airline)
    df_airports = spark.read.parquet(read_parquet_airports)
except Exception as error:
    logger.error("Error read.parquet:" + str(error))

14:53:21.882 [Thread-3] INFO  org.apache.spark.sql.internal.SharedState - Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
14:53:21.892 [Thread-3] INFO  org.apache.spark.sql.internal.SharedState - Warehouse path is 'file:/home/illidan/proyecto_desde0/ETL/spark-warehouse'.
14:53:21.914 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@4b7b3a0{/SQL,null,AVAILABLE,@Spark}
14:53:21.915 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@4967ea8{/SQL/json,null,AVAILABLE,@Spark}
14:53:21.916 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@774d3745{/SQL/execution,null,AVAILABLE,@Spark}
14:53:21.916 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@1ded8a2c{/SQL/execution/json,null,AVAILABLE,@Spark}
14:53:21.928 [Thread-

14:53:24.830 [Executor task launch worker for task 0.0 in stage 0.0 (TID 0)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 0.0 (TID 0). 3053 bytes result sent to driver
14:53:24.843 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 0.0 in stage 0.0 (TID 0) in 594 ms on 172.23.57.81 (executor driver) (1/1)
14:53:24.844 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSchedulerImpl - Removed TaskSet 0.0, whose tasks have all completed, from pool 
14:53:24.849 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - ResultStage 0 (parquet at NativeMethodAccessorImpl.java:0) finished in 0.814 s
14:53:24.852 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Job 0 is finished. Cancelling potential speculative or zombie tasks for this job
14:53:24.853 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.TaskSchedulerImpl - Killing all running tasks in stage 0: Stage finish

14:53:25.448 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_0_piece0 on 172.23.57.81:37441 in memory (size: 36.9 KiB, free: 434.4 MiB)
14:53:25.745 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.InMemoryFileIndex - It took 3 ms to list leaf files for 1 paths.
14:53:25.788 [Thread-3] INFO  org.apache.spark.SparkContext - Starting job: parquet at NativeMethodAccessorImpl.java:0
14:53:25.789 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Got job 1 (parquet at NativeMethodAccessorImpl.java:0) with 1 output partitions
14:53:25.789 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Final stage: ResultStage 1 (parquet at NativeMethodAccessorImpl.java:0)
14:53:25.789 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Parents of final stage: List()
14:53:25.789 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Missing parents: Lis

In [6]:
try:
    #avg per fly
    avg_flight = df_flights.join(broadcast(df_airline), df_flights.AIRLINE == df_airline.IATA_CODE) \
                        .groupBy(df_flights.AIRLINE, df_airline.AIRLINE) \
                        .agg(round(avg(df_flights.DISTANCE), 2).alias("avg_DISTANCE")) \
                        .orderBy(desc("avg_DISTANCE")) \
                        .select(df_airline.AIRLINE.alias("AIRLINE_Name"), "avg_DISTANCE")
except Exception as error:
    logger.error("Error variable avg_flight:" + str(error))


In [ ]:

try:
    #how many times a flight visit an airport
    airline_in_airport = df_flights.join(broadcast(df_airports), df_flights.ORIGIN_AIRPORT == df_airports.IATA_CODE) \
                                .join(broadcast(df_airline), df_flights.AIRLINE == df_airline.IATA_CODE) \
                                .repartition(df_flights.DESTINATION_AIRPORT) \
                                .groupBy(df_flights.DESTINATION_AIRPORT, df_airline.AIRLINE) \
                                    .agg(count(df_flights.AIRLINE).alias("Count_visit_per_airline")) \
                                .orderBy(asc(df_flights.DESTINATION_AIRPORT)) \
                                .select(df_flights.DESTINATION_AIRPORT, df_airline.AIRLINE,"Count_visit_per_airline")
except Exception as error:
    logger.error("Error variable airline_in_airport:" + str(error))

#a Rank of wich airline is the most visited in each airport
try:
    window_spec = Window.partitionBy("DESTINATION_AIRPORT").orderBy(desc("Count_visit_per_airline"))
    
    Rank_airline_in_airport = airline_in_airport.withColumn("Ranking", rank().over(window_spec))


except Exception as error:
    logger.error("Error Rank_airline_in_airport:" + str(error))


In [8]:
try:
    #mention how many times an airline visit a destination and canceled this arrival
    flights_per_cancell = df_flights.select(col("AIRLINE"), col("CANCELLED"), col("DESTINATION_AIRPORT")) \
                                .filter(col("CANCELLED") == "1") \
                                .repartition(col("AIRLINE"), col("DESTINATION_AIRPORT")) \
                                .groupBy(col("AIRLINE"), col("DESTINATION_AIRPORT")) \
                                    .agg(count(col("AIRLINE")).alias("count_airlines_cancel")) \
                                .orderBy(desc("DESTINATION_AIRPORT")) \
                                .limit(100)
except Exception as error:
    logger.error("Error variable flights_per_cancell:" + str(error))


In [9]:
try:
    #i create this variable because i need to know how many airports have flights but are not in the list of airports
    airport_notin_thelist = df_flights.join(broadcast(df_airports), df_flights.DESTINATION_AIRPORT == df_airports.IATA_CODE, "left_anti") \
                                        .select("DESTINATION_AIRPORT") \
                                        .repartition("DESTINATION_AIRPORT") \
                                        .groupBy("DESTINATION_AIRPORT").agg(count("*").alias("Count_airports"))
except Exception as error:
    logger.error("Error variable airport_notin_thelist:" + str(error))


In [10]:
#Create parquet files

try:
    avg_flight.write.parquet(config["Parquet_file"]["avg_flight"], mode="overwrite")

    Rank_airline_in_airport.write.parquet(config["Parquet_file"]["airline_in_airport"], mode="overwrite")

    flights_per_cancell.write.parquet(config["Parquet_file"]["flights_per_cancell"], mode="overwrite")

    airport_notin_thelist.write.parquet(config["Parquet_file"]["airport_notin_thelist"], mode="overwrite")
except Exception as error:
    logger.error("Error write parquet files:" + str(error))


14:53:29.735 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_10_piece0 on 172.23.57.81:37441 in memory (size: 7.8 KiB, free: 434.3 MiB)
14:53:29.741 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_11_piece0 on 172.23.57.81:37441 in memory (size: 5.9 KiB, free: 434.3 MiB)
14:53:29.757 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Pushed Filters: IsNotNull(AIRLINE)
14:53:29.758 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Post-Scan Filters: isnotnull(AIRLINE#4)
14:53:29.762 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Pushed Filters: IsNotNull(IATA_CODE)
14:53:29.762 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Post-Scan Filters: isnotnull(IATA_CODE#62)
14:53:29.868 [broadcast-exchange-0] INFO  org.apache.spark.sql.catalyst.expressions.codegen.C

14:53:33.024 [Executor task launch worker for task 5.0 in stage 13.0 (TID 22)] INFO  org.apache.spark.executor.Executor - Finished task 5.0 in stage 13.0 (TID 22). 3926 bytes result sent to driver
14:53:33.024 [Executor task launch worker for task 6.0 in stage 13.0 (TID 23)] INFO  org.apache.spark.executor.Executor - Finished task 6.0 in stage 13.0 (TID 23). 3926 bytes result sent to driver
14:53:33.026 [task-result-getter-1] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 6.0 in stage 13.0 (TID 23) in 2292 ms on 172.23.57.81 (executor driver) (1/8)
14:53:33.026 [task-result-getter-3] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 5.0 in stage 13.0 (TID 22) in 2292 ms on 172.23.57.81 (executor driver) (2/8)
14:53:33.029 [Executor task launch worker for task 2.0 in stage 13.0 (TID 19)] INFO  org.apache.spark.executor.Executor - Finished task 2.0 in stage 13.0 (TID 19). 3926 bytes result sent to driver
14:53:33.032 [task-result-getter-2] INFO  org.apache.

14:53:33.262 [Thread-3] INFO  org.apache.spark.sql.catalyst.expressions.codegen.CodeGenerator - Code generated in 26.535816 ms
14:53:33.319 [Thread-3] INFO  org.apache.spark.SparkContext - Starting job: parquet at NativeMethodAccessorImpl.java:0
14:53:33.320 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Got job 11 (parquet at NativeMethodAccessorImpl.java:0) with 1 output partitions
14:53:33.320 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Final stage: ResultStage 15 (parquet at NativeMethodAccessorImpl.java:0)
14:53:33.320 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Parents of final stage: List(ShuffleMapStage 14)
14:53:33.321 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Missing parents: List()
14:53:33.322 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Submitting ResultStage 15 (MapPartitionsRDD[39] at parquet at NativeMethodAccessorImpl.

14:53:33.463 [Executor task launch worker for task 0.0 in stage 17.0 (TID 26)] INFO  org.apache.spark.executor.Executor - Running task 0.0 in stage 17.0 (TID 26)
14:53:33.495 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_16_piece0 on 172.23.57.81:37441 in memory (size: 23.2 KiB, free: 434.3 MiB)
14:53:33.511 [Executor task launch worker for task 0.0 in stage 17.0 (TID 26)] INFO  org.apache.spark.sql.catalyst.expressions.codegen.CodeGenerator - Code generated in 19.485719 ms
14:53:33.521 [Executor task launch worker for task 0.0 in stage 17.0 (TID 26)] INFO  org.apache.spark.storage.ShuffleBlockFetcherIterator - Getting 8 (11.2 KiB) non-empty blocks including 8 (11.2 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) push-merged-local and 0 (0.0 B) remote blocks
14:53:33.522 [Executor task launch worker for task 0.0 in stage 17.0 (TID 26)] INFO  org.apache.spark.storage.ShuffleBlockFetcherIterator - Started 0 remote fetches in 2 ms
14

14:53:35.190 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_12_piece0 on 172.23.57.81:37441 in memory (size: 34.6 KiB, free: 434.2 MiB)
14:53:35.247 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_14_piece0 on 172.23.57.81:37441 in memory (size: 686.0 B, free: 434.3 MiB)
14:53:35.312 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_15_piece0 on 172.23.57.81:37441 in memory (size: 34.7 KiB, free: 434.3 MiB)
14:53:38.767 [Executor task launch worker for task 6.0 in stage 23.0 (TID 36)] INFO  org.apache.spark.executor.Executor - Finished task 6.0 in stage 23.0 (TID 36). 2451 bytes result sent to driver
14:53:38.773 [task-result-getter-3] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 6.0 in stage 23.0 (TID 36) in 4191 ms on 172.23.57.81 (executor driver) (1/8)


...

14:53:42.480 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileFormatWriter - Write Job b1b5a451-3bf7-4524-bcfe-b9eb73a66bdf committed. Elapsed time: 30 ms.
14:53:42.481 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileFormatWriter - Finished processing stats for write job b1b5a451-3bf7-4524-bcfe-b9eb73a66bdf.
14:53:42.537 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Pushed Filters: IsNotNull(CANCELLED),EqualTo(CANCELLED,1)
14:53:42.537 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Post-Scan Filters: isnotnull(CANCELLED#24),(CANCELLED#24 = 1)
14:53:42.583 [Thread-3] INFO  org.apache.spark.sql.catalyst.expressions.codegen.CodeGenerator - Code generated in 13.749364 ms
14:53:42.587 [Thread-3] INFO  org.apache.spark.storage.memory.MemoryStore - Block broadcast_31 stored as values in memory (estimated size 200.5 KiB, free 429.1 MiB)
14:53:42.597 [Thread-3] INFO  org.apache.spark.storage.memo

14:53:43.367 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 0.0 in stage 31.0 (TID 59) in 748 ms on 172.23.57.81 (executor driver) (4/8)
14:53:43.454 [Executor task launch worker for task 5.0 in stage 31.0 (TID 64)] INFO  org.apache.spark.executor.Executor - Finished task 5.0 in stage 31.0 (TID 64). 2296 bytes result sent to driver
14:53:43.455 [task-result-getter-3] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 5.0 in stage 31.0 (TID 64) in 834 ms on 172.23.57.81 (executor driver) (5/8)
14:53:43.456 [Executor task launch worker for task 2.0 in stage 31.0 (TID 61)] INFO  org.apache.spark.executor.Executor - Finished task 2.0 in stage 31.0 (TID 61). 2296 bytes result sent to driver
14:53:43.458 [task-result-getter-2] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 2.0 in stage 31.0 (TID 61) in 838 ms on 172.23.57.81 (executor driver) (6/8)
14:53:43.462 [Executor task launch worker for task 3.0 in stage 31.0 (TID 6

14:53:44.032 [Executor task launch worker for task 0.0 in stage 34.0 (TID 68)] INFO  org.apache.spark.executor.Executor - Running task 0.0 in stage 34.0 (TID 68)
14:53:44.037 [Executor task launch worker for task 0.0 in stage 34.0 (TID 68)] INFO  org.apache.spark.sql.execution.datasources.FileScanRDD - Reading File path: file:///home/illidan/proyecto_desde0/archivos_parquet/df_airports/part-00000-10922ef7-e1bf-447c-b966-c26b9b507f33-c000.snappy.parquet, range: 0-17990, partition values: [empty row]
14:53:44.044 [Executor task launch worker for task 0.0 in stage 34.0 (TID 68)] INFO  org.apache.parquet.filter2.compat.FilterCompat - Filtering using predicate: noteq(IATA_CODE, null)
14:53:44.049 [Executor task launch worker for task 0.0 in stage 34.0 (TID 68)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 34.0 (TID 68). 3613 bytes result sent to driver
14:53:44.051 [task-result-getter-2] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 0.0 in stage 34

14:53:44.685 [Executor task launch worker for task 0.0 in stage 35.0 (TID 69)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 35.0 (TID 69). 2210 bytes result sent to driver
14:53:44.687 [task-result-getter-1] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 0.0 in stage 35.0 (TID 69) in 552 ms on 172.23.57.81 (executor driver) (1/8)
14:53:44.693 [Executor task launch worker for task 3.0 in stage 35.0 (TID 72)] INFO  org.apache.spark.executor.Executor - Finished task 3.0 in stage 35.0 (TID 72). 2210 bytes result sent to driver
14:53:44.699 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 3.0 in stage 35.0 (TID 72) in 563 ms on 172.23.57.81 (executor driver) (2/8)
14:53:44.702 [Executor task launch worker for task 2.0 in stage 35.0 (TID 71)] INFO  org.apache.spark.executor.Executor - Finished task 2.0 in stage 35.0 (TID 71). 2210 bytes result sent to driver
14:53:44.703 [task-result-getter-3] INFO  org.apache.sp

14:53:44.944 [Executor task launch worker for task 6.0 in stage 35.0 (TID 75)] INFO  org.apache.spark.executor.Executor - Finished task 6.0 in stage 35.0 (TID 75). 2339 bytes result sent to driver
14:53:44.945 [task-result-getter-2] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 6.0 in stage 35.0 (TID 75) in 806 ms on 172.23.57.81 (executor driver) (8/8)
14:53:44.945 [task-result-getter-2] INFO  org.apache.spark.scheduler.TaskSchedulerImpl - Removed TaskSet 35.0, whose tasks have all completed, from pool 
14:53:44.945 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - ShuffleMapStage 35 (parquet at NativeMethodAccessorImpl.java:0) finished in 0.819 s
14:53:44.946 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - looking for newly runnable stages
14:53:44.946 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - running: Set()
14:53:44.946 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DA